导读
R-Breaker 是一种短线日内交易策略，它结合了趋势和反转两种交易方式。

本文给出了趋势和反转的策略示例，展示信号出现时的操作。

(优矿策略库版权归优矿所有，禁止直接转载或编辑后转载。)

In [ ]:
import numpy as np
import pandas as pd
import talib as ta
from collections import deque
from itertools import product
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

# <font color=#0099ff> 1. R-Breaker策略描述 </font>
---

R-Breaker 是一种短线日内交易策略，它结合了趋势和反转两种交易方式。该策略也长期被Future Thruth 杂志评为最赚钱的策略之一，尤其在标普500 股指期货上效果最佳。该策略的主要特点如下：

第一、根据前一个交易日的收盘价、最高价和最低价数据通过一定方式计算出六个价位，从大到小依次为突破买入价、观察卖出价、反转卖出价、反转买入价、观察买入价和突破卖出价，以此来形成当前交易日盘中交易的触发条件。通过对计算方式的调整，可以调节六个价格间的距离，进一步改变触发条件。

第二、根据盘中价格走势，实时判断触发条件，具体条件如下：
	1) 当日内最高价超过观察卖出价后，盘中价格出现回落，且进一步跌破反转卖出价构成的支撑线时，采取反转策略，即在该点位（反手、开仓）做空；
	2) 当日内最低价低于观察买入价后，盘中价格出现反弹，且进一步超过反转买入价构成的阻力线时，采取反转策略，即在该点位（反手、开仓）做多；
	3) 在空仓的情况下，如果盘中价格超过突破买入价，则采取趋势策略，即在该点位开仓做多；
	4) 在空仓的情况下，如果盘中价格跌破突破卖出价，则采取趋势策略，即在该点位开仓做空。

第三、设定止损以及止盈条件；

第四、设定过滤条件；

第五、在每日收盘前，对所持合约进行平仓。

具体来看，这六个价位形成的阻力和支撑位计算过程如下：
* 观察卖出价 = High + 0.35 \* (Close – Low)
* 观察买入价 = Low – 0.35 \* (High – Close)
* 反转卖出价 = 1.07 / 2 \* (High + Low) – 0.07 * Low
* 反转买入价 = 1.07 / 2 \* (High + Low) – 0.07 * High
* 突破买入价 = 观察卖出价 + 0.25 * (观察卖出价 – 观察买入价)
* 突破卖出价 = 观察买入价 – 0.25 * (观察卖出价 – 观察买入价)

其中，High、Close、Low 分别为昨日最高价、昨日收盘价和昨日最低价。这六个价位从大到小依次是，突破买入价、观察卖出价、反转卖出价、反转买入价、观察买入价和突破卖出价。

# <font color=#0099ff> 2. 策略源码 </font>

In [ ]:
universe = ['RBM0']                  # 策略交易的期货合约，此处选择IH1609
start = "2017-01-01"                 # 回测开始时间
end   = "2017-12-31"                 # 回测结束时间
refresh_rate = 5                    # 算法调用周期
freq = 'm'                           # 算法调用频率：m-> 分钟；d-> 日；

commission = {'RB': (0.000025, 'perValue')}
slippage = Slippage(0, 'perValue')
accounts = {
    'fantasy_account': AccountConfig(account_type='futures', capital_base=100000, commission=commission, slippage=slippage)
}

amount = 20

def initialize(context):
    context.high = np.NAN
    context.low = np.NAN
    context.close = np.NAN
    context.count1 = 0
    context.count2 = 0
    
def handle_data(context):
    symbol = context.get_symbol('RBM0')
    
    futures_account = context.get_account('fantasy_account')
    try:
        long_position = futures_account.get_position(symbol).long_amount
    except:
        long_position = 0
        
    try:
        short_position = futures_account.get_position(symbol).short_amount
    except:
        short_position = 0
        
    current_time = context.now.strftime('%H:%M:%S')
    yesterday = context.previous_date.strftime('%Y-%m-%d')
    # print context.now
    
    if current_time == '09:00:00':    
        kline_day = context.history(symbol='RBM0', attribute=[u'closePrice', u'highPrice',u'lowPrice'], time_range=1, freq='1d', style='sat')['RBM0']
        
        # print kline_day.loc[yesterday]
        context.high = kline_day['highPrice'][-1]
        context.low = kline_day['lowPrice'][-1]
        context.close = kline_day['closePrice'][-1]

    if (current_time > '09:00:00' and current_time < '11:30:00') or (current_time > '13:30:00' and current_time < '14:55:00'):     
        kline_5m = context.history(symbol='RBM0', attribute=['openPrice', 'highPrice', 'lowPrice', 'closePrice'], time_range=2, freq='5m', style='sat')['RBM0']
        
        before_2_close = kline_5m['closePrice'][-2]
        before_2_high = kline_5m['highPrice'][-2]   
        before_1_close = kline_5m['closePrice'][-1]
        before_1_high = kline_5m['highPrice'][-1]
        before_1_low = kline_5m['lowPrice'][-1]
        before_1_open = kline_5m['openPrice'][-1]
        # print before_2_close
        
        # 观察卖出价
        ssetup = context.high + 0.35*(context.close-context.low)
        # 观察买入价
        bsetup = context.low - 0.35*(context.high-context.close)
        # 反转卖出价
        senter = (1+0.07)/2 * (context.high+context.low)-0.07*context.low
        # 反转买入价
        benter = (1+0.07)/2 * (context.high+context.low)-0.07*context.high
        # 突破买入价
        bbreak = ssetup+0.25 * (ssetup-bsetup)
        # 突破卖出价
        sbreak = bsetup-0.25 * (ssetup-bsetup)

        ## 趋势
        if before_2_close <= bbreak and before_1_close > bbreak:
            if long_position == 0:
                print context.now, '趋势向上突破，买入开仓', amount
                futures_account.order(symbol, amount, 'open')
            if short_position != 0:
                print context.now, '趋势向上突破，买入平仓', short_position
                futures_account.order(symbol, short_position, 'close')
        if before_2_close >= sbreak and before_1_close < sbreak:
            if short_position == 0:
                print context.now, '趋势向下突破，卖出开仓', amount
                futures_account.order(symbol, -amount, 'open')
            if long_position != 0:
                print context.now, '趋势向下突破，卖出平仓', long_position
                futures_account.order(symbol, -long_position, 'close')
                
        # 反转
        ## 多单反转
        if before_1_high > ssetup and before_1_close > senter:
            context.count1 = 1
        if context.count1 == 1 and before_1_close < senter:
            # print context.now, '趋势向下反转，卖出', long_position
            if long_position > 0:
                print context.now, '趋势向下反转，卖出平仓', long_position, '买入开仓', amount
                futures_account.order(symbol, -long_position, 'close')
                futures_account.order(symbol, -amount, 'open')
                
        ### 空单反转
        if before_1_low < bsetup:
            context.count2 = 1
        if context.count2 == 1 and before_1_close > benter:
            # print '趋势向上反转，买入'
            if short_position != 0:
                print context.now, '趋势向上反转，买入平仓', short_position, '卖出开仓', amount
                futures_account.order(symbol, short_position, 'close')
                futures_account.order(symbol, amount, 'open')           

    elif current_time >= '14:55:00':
        if futures_account.get_positions():
            futures_account.close_all_positions()
            
        context.high = np.NAN
        context.low = np.NAN
        context.close = np.NAN
        context.count1 = 0
        context.count2 = 0